In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport notes
%aimport watch_point
%aimport cluster
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from pprint import pprint
import math
import constants as const

In [2]:
import pickle

def load_phrase_base(file_name: str) -> (dict, list):
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
        phrase_base = data.get('phrase_base', {})
        marks = data.get('marks', {})
        return phrase_base, marks
    
phrase_base, marks = load_phrase_base('./data/texts/phrase_base.pickle')

In [4]:
from ipythonblocks import BlockGrid

def draw_notation(notation: np.array):
    bit_grid = BlockGrid(len(notation), 1, fill=(17, 41, 129))
    for block in range(bit_grid.width):
        color = bit_grid[0, block]
        if notation[block]:
            bit_grid[0, block] = (244, 195, 173)
    bit_grid.lines_on = False
    bit_grid.show()

def draw_note_notations(notes, note_idx: int):
    for i in range(len(notes[note_idx])):
        draw_notation(notes.note_notation_as_bits(note_idx, i))

In [5]:
abc_notes = notes.Notes(note_count=26, 
                        notation_count=10, 
                        active_bits=8, 
                        bit_count=255)

key = list(phrase_base.keys())[0]
phrase = phrase_base[key][0]
bit_chord = abc_notes.phrase_chord(phrase)
draw_notation(abc_notes.notation_as_bits(bit_chord))
print(phrase)
for note_idx, notation_idx in phrase:
    notation = abc_notes.note_notation_as_bits(note_idx, notation_idx)
    draw_notation(notation)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


[(8, 0), (0, 1), (13, 2), (24, 3), (18, 4)]


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [4]:
feeder = PhraseFeeder(phrase_base, marks)    

In [ ]:
phrases, output_bits = feeder.take_phrase_bunch()
print(output_bits, marks[output_bits])
print(phrases)

In [3]:
def feed_phrase_batch(feeder, count=200):
    phrases, output_bits = feeder.take_phrase_batch(count=200)
    output_bits = set(output_bits)
    for phrase in phrases:
        bit_chord = abc_notes.phrase_chord(phrase)
        cxnn.receive_bits(input_bits=bit_chord, output_bits=output_bits)

def feed_n_batches(feeder, n=10, batch_size=200):
    for i in range(n):
        feed_phrase_batch(feeder, count=batch_size)
        print(f'batch {i+1}/{n}')
    print(f'clusters: {cxnn.cluster_count()}')
        
def show_point_stats(cxnn: ContextNN):
    cluster_counts = [wp.cluster_count() for wp in cxnn.point_objects]
    output_bits = [(wp.output_bit, wp.cluster_count()) 
                   for wp in cxnn.point_objects]
    print('Cluster count:', sum(cluster_counts))
    print(output_bits)
  

In [54]:
abc_notes = notes.Notes(note_count=26, 
                        notation_count=10, 
                        active_bits=8, 
                        bit_count=255)

feeder = phrase_feeder.PhraseFeeder(phrase_base, marks)  

cxnn = context_nn.ContextNN(input_bit_count=255,
                            output_bit_count=8,
                            watch_point_count=100,
                            watch_bit_count=32,
                            cluster_make_threshold=6,
                            cluster_activate_threshold=4)

In [39]:
cxnn.state = const.STATE_LEARN

In [35]:
cxnn.state = const.STATE_CONSOLIDATE

In [79]:
%%time
feed_n_batches(feeder, n=30, batch_size=200)

batch 1/30
batch 2/30
batch 3/30
batch 4/30
batch 5/30
batch 6/30
batch 7/30
batch 8/30
batch 9/30
batch 10/30
batch 11/30
batch 12/30
batch 13/30
batch 14/30
batch 15/30
batch 16/30
batch 17/30
batch 18/30
batch 19/30
batch 20/30
batch 21/30
batch 22/30
batch 23/30
batch 24/30
batch 25/30
batch 26/30
batch 27/30
batch 28/30
batch 29/30
batch 30/30
clusters: 94891
Wall time: 1min 11s


In [80]:
%%time
cxnn.reduce_clusters(min_component=0.2, 
                     min_activations=50,
                     trim=True,
                     remain_part=0.3,
                     clear_stats=True,
                     consolidate=True,
                     amnesty=True)

Wall time: 2.66 s


In [81]:
show_point_stats(cxnn)
marks

Cluster count: 35528
[(2, 84), (3, 429), (3, 484), (7, 95), (7, 216), (6, 790), (0, 798), (2, 216), (6, 520), (7, 295), (0, 576), (4, 60), (5, 277), (0, 599), (2, 8), (5, 668), (5, 311), (2, 163), (0, 773), (4, 39), (2, 43), (7, 447), (1, 640), (5, 284), (4, 75), (5, 726), (2, 42), (3, 341), (2, 19), (3, 359), (0, 649), (7, 451), (5, 411), (2, 222), (5, 284), (6, 648), (2, 187), (1, 559), (2, 171), (3, 158), (0, 1014), (5, 316), (7, 74), (0, 693), (6, 359), (7, 131), (2, 196), (5, 294), (2, 228), (5, 110), (1, 725), (4, 149), (5, 326), (7, 366), (4, 19), (4, 60), (5, 80), (1, 438), (2, 76), (4, 102), (5, 331), (5, 301), (0, 764), (3, 460), (3, 149), (4, 105), (4, 273), (5, 137), (5, 197), (5, 498), (7, 818), (5, 519), (3, 396), (3, 117), (7, 86), (4, 97), (5, 414), (1, 988), (7, 118), (4, 51), (7, 225), (4, 158), (2, 142), (1, 467), (5, 296), (2, 91), (5, 634), (4, 153), (5, 203), (6, 1250), (7, 344), (4, 182), (7, 527), (1, 648), (3, 374), (6, 1011), (0, 737), (0, 316), (0, 622), (7, 

{(0, 1, 3, 5): 'ukr',
 (0, 1, 6, 7): 'blg',
 (0, 3, 5, 6): 'epo',
 (0, 5, 6, 7): 'bel',
 (1, 2, 3, 6): 'pol',
 (1, 2, 4, 7): 'jbo',
 (1, 4, 6, 7): 'eng',
 (2, 3, 4, 5): 'rus'}

In [66]:
def dict_value(d: dict, n=0) -> object:
    i = 0
    diter = iter(d)
    while i <= n:
        key = next(diter)
        i += 1
    return d[key]


point = dict_value(cxnn.watch_points, n=0)
cluster_idx = 0
cluster = point.cluster_objects[cluster_idx]
pprint(cluster.stats)
print(sum(cluster.stats.values()))
# pprint(cluster.component_stats())
# pprint(cluster.bit_rate())

pprint(sorted(cluster.component_stats().items(), key=lambda x: x[1], reverse=True))
print(point.cluster_masks)
print(tuple(sorted(cluster.bits)))
print(point.watch_bits)
print(point.cluster_masks[cluster_idx])
print(len(point.cluster_masks), len(point.cluster_objects))

# parts = cluster.component_s
# enthropy = 0.0
# for part in parts.values():
#     enthropy += -part * math.log2(part)
# print(enthropy)

{}
0
[]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 1 1]
 ..., 
 [0 0 0 ..., 1 0 1]
 [1 0 0 ..., 0 0 0]
 [0 0 0 ..., 1 0 0]]
(25, 39, 92, 145, 147)
(1, 2, 3, 14, 25, 27, 39, 42, 43, 47, 53, 56, 73, 88, 92, 101, 145, 147, 160, 162, 167, 173, 177, 198, 199, 204, 208, 213, 223, 231, 240, 252)
[0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
32 32


In [ ]:
noize_clusters = 0
total_clusters = 0
cluster_lens = {}
cluster_acts = {}
for wp in cxnn.watch_points.values():
    for cluster in wp.cluster_objects:
        total_clusters += 1
        cluster_lens[len(cluster.bits)] = cluster_lens.get(len(cluster.bits), 0) + 1
        cluster_acts[len(cluster.stats)] = cluster_acts.get(len(cluster.stats), 0) + 1
        if not cluster.has_big_component(threshold=0.15, min_activations=15):
            noize_clusters += 1
print(total_clusters)
print(noize_clusters)
pprint(cluster_lens)
pprint(cluster_acts)


In [82]:
def consilidated_stats(cxnn: ContextNN):
    stats = dict()
    for wp in cxnn.point_objects:
        for cluster in wp.cluster_objects:    
            stats[cluster.consolidated] = stats.get(cluster.consolidated, 0) + 1
    return stats

consilidated_stats(cxnn)
    

{0: 4997, 1: 13464, 2: 6743, 3: 4124, 4: 3329, 5: 2871}

In [24]:
cluster_with_stats = None
for wp in cxnn.point_objects:
    for cluster in wp.cluster_objects:    
        if cluster.stats:
            cluster_with_stats = cluster
            break
    else: 
        continue
    break    

print(cluster_with_stats.stats, cluster_with_stats.consolidated)
    

{(19, 39, 110, 117, 226): 1} 0


In [26]:
my_cluster = Cluster((10, 20, 30), bitarray(10), 4)
print(type(my_cluster))
my_cluster.bits
my_cluster.stat_parts()

<class 'cluster.Cluster'>


{}

In [6]:
[(8, 0), (0, 1), (13, 2), (24, 3), (18, 4)].__sizeof__()

80